In [1]:
from pyspark.sql import SparkSession
import os
import time

In [2]:
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'

In [3]:
print("Starting Unity Catalog Verification...")
spark_master = os.getenv("SPARK_MASTER", "local[*]")
uc_uri = os.getenv("UC_URI", "http://localhost:8081")
spark_catalog_name = os.getenv("SPARK_CATALOG_NAME", "spark_catalog")
schema_name = os.getenv("SPARK_SCHEMA_NAME", "default_schema")
table_name = os.getenv("SPARK_TEST_TABLE", f"test_table_{int(time.time())}")
spark_shared_location = os.getenv("SPARK_SHARED_LOCATION", "")
scala_binary = os.getenv("SCALA_BINARY_VERSION", "2.12")
uc_version = os.getenv("UC_SPARK_VERSION", "0.2.1")
delta_version = os.getenv("DELTA_SPARK_VERSION", "3.2.1")
scala_library_version = os.getenv("SCALA_LIBRARY_VERSION", "2.12.18")
spark_packages = ",".join([
    f"io.unitycatalog:unitycatalog-spark_{scala_binary}:{uc_version}",
    f"io.delta:delta-spark_{scala_binary}:{delta_version}",
    f"org.scala-lang:scala-library:{scala_library_version}",
])

Starting Unity Catalog Verification...


In [4]:
builder = SparkSession.builder \
    .appName("UnityCatalogVerification") \
    .master(spark_master) \
    .config("spark.jars.packages", spark_packages) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog") \
    .config("spark.sql.catalog.spark_catalog.uri", uc_uri) \
    .config("spark.sql.catalog.spark_catalog.token", "") \
    .config("spark.sql.defaultCatalog", spark_catalog_name)

spark = builder.getOrCreate()

:: loading settings :: url = jar:file:/Users/surendrashukla/projects/spark-unity/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/surendrashukla/.ivy2/cache
The jars for the packages stored in: /Users/surendrashukla/.ivy2/jars
io.unitycatalog#unitycatalog-spark_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.scala-lang#scala-library added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0e9bcab-f53c-4ddc-b1b5-366bd9e23ef9;1.0
	confs: [default]
	found io.unitycatalog#unitycatalog-spark_2.12;0.2.1 in central
	found io.unitycatalog#unitycatalog-client;0.2.1 in central
	found org.slf4j#slf4j-api;2.0.13 in central
	found org.apache.logging.log4j#log4j-slf4j2-impl;2.23.1 in central
	found org.apache.logging.log4j#log4j-api;2.23.1 in central
	found org.apache.logging.log4j#log4j-core;2.23.1 in central
	found com.fasterxml.jackson.datatype#jackson-datatype-jsr310;2.17.0 in central
	found org.openapitools#jackson-databind-nullable;0.2.6 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.fasterxml.ja

In [5]:
spark.sql("SHOW SCHEMAS").show()

+--------------+
|     namespace|
+--------------+
|default_global|
|default_schema|
+--------------+



In [6]:
spark.sql("CREATE SCHEMA IF NOT EXISTS spark_catalog.default_schema")

DataFrame[]

In [7]:
spark.sql("USE spark_catalog.default_schema")

DataFrame[]

In [8]:
spark.sql("SELECT current_catalog(), current_schema()").show(truncate=False)

+-----------------+------------------+
|current_catalog()|current_database()|
+-----------------+------------------+
|spark_catalog    |default_schema    |
+-----------------+------------------+



In [10]:
import time
table = f"spark_catalog.default_schema.customers_{int(time.time())}"
location = f"/tmp/uc_nb_{int(time.time())}"

spark.sql(f"""
CREATE TABLE {table} (
  customer_id INT,
  first_name STRING,
  last_name STRING,
  email STRING,
  created_at TIMESTAMP
) USING DELTA
LOCATION '{location}'
""")

spark.sql(f"INSERT INTO {table} VALUES (1,'A','B','a@x.com', current_timestamp())")
spark.sql(f"SELECT * FROM {table}").show(truncate=False)


26/02/21 10:29:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+-----------+----------+---------+-------+--------------------------+
|customer_id|first_name|last_name|email  |created_at                |
+-----------+----------+---------+-------+--------------------------+
|1          |A         |B        |a@x.com|2026-02-21 10:29:39.707466|
+-----------+----------+---------+-------+--------------------------+

